In [1]:
from __future__ import absolute_import
from six.moves import cPickle
import gzip
import random
import numpy as np

import glob, os, csv, re
from collections import Counter
from keras.preprocessing import sequence
import operator

Using TensorFlow backend.


In [2]:
def init_vocab(filelist="list_of_grid.txt", occur=30):
    
    wordcount={}
    list_of_files = [line.rstrip('\n') for line in open(filelist)]
    print(len(list_of_files))
    #print(list_of_files)
    
    
    for file in list_of_files:
        #print(file) 
        lines = [line.rstrip('\n') for line in open(file + ".EGrid")]
        #print(lines)
        for line in lines:
            ent = line.split()[0]
            if ent not in wordcount:
                wordcount[ent] = 1
            else:
                wordcount[ent] += 1
    #print(len(wordcount))
    #print sorted(wordcount.items(), key=lambda x: x[1])
    

    wordlist = sorted(wordcount.items(), key=operator.itemgetter(1), reverse=True)    # sorted by value
    print("Number of entities: ", len(wordlist))
    #print(wordlist)
    n = (int)(len(wordlist)*occur/100)
    print("Using", n, "for training as", occur, "percentage")

    wordlist = wordlist[0:n]
    vocabs = []
    
    for tup in wordlist:
        vocabs.append(tup[0])    
    vocabs.append("oov") # of our vocabulary word

    return vocabs

In [3]:
vocabs = init_vocab(filelist="wsj.train_dev")
len(vocabs)
vocabs

1378
Number of entities:  19903
Using 5970 for training as 30 percentage


['%',
 'COMPANY',
 'YEAR',
 'MR.',
 'MILLION',
 'CORP.',
 'INC.',
 'NEW',
 'YEARS',
 'PRESIDENT',
 'MARKET',
 'CO.',
 'STOCK',
 'U.S.',
 'YORK',
 'SHARE',
 'GROUP',
 'BUSINESS',
 'TIME',
 'THERE',
 'EXCHANGE',
 'ONE',
 'SHARES',
 'YESTERDAY',
 'TRADING',
 'CHAIRMAN',
 'MONTHS',
 'SALES',
 'PEOPLE',
 'THAT',
 'COMPANIES',
 '$',
 'UNIT',
 'GOVERNMENT',
 'PRICE',
 'WEEK',
 'BILLION',
 'INTEREST',
 'PART',
 'OFFICIALS',
 'MONTH',
 'INDUSTRY',
 'THIS',
 'CENTS',
 'BANK',
 'INVESTMENT',
 'BOARD',
 'CONCERN',
 'QUARTER',
 'SECURITIES',
 'VICE',
 'SOME',
 'ABOUT',
 'MONEY',
 'ANALYSTS',
 'WORLD',
 'EARNINGS',
 'EXECUTIVE',
 'INCOME',
 'PRICES',
 'PRODUCTS',
 'INVESTORS',
 'PLAN',
 'RATE',
 'AMERICAN',
 'CAPITAL',
 'DIRECTOR',
 'OPERATIONS',
 'WAY',
 'NUMBER',
 'MARKETS',
 'END',
 'OFFICER',
 'STATE',
 'SALE',
 'SPOKESMAN',
 'PROGRAM',
 'DAYS',
 'TODAY',
 'REVENUE',
 'OCT.',
 'VALUE',
 'PROFIT',
 'SERVICES',
 'AGREEMENT',
 'DAY',
 'MAKER',
 'INTERNATIONAL',
 'NATIONAL',
 'MANAGEMENT',
 'PERIOD'

In [4]:
def get_eTrans_with_Word(sent, vocabs):
    x = sent.split()
    #print(x)
    length = len(x)
    #print(length)
    e_occur = x.count('X') + x.count('S') + x.count('O') #counting the number of occurrence of entities
    #print(e_occur)
    if length > 80:
        if e_occur < 3:
            return ""
    elif length > 20:
        if e_occur < 2:
            return ""
    
    ent = x[0]
    #print(ent)
    x = x[1:]
    
    new_x = []

    for role in x:
        if role != "-":
            if ent in vocabs:
                new_x.append(ent+"_"+role)
            else:
                new_x.append("ovv_"+ role)
                
        else:
            new_x.append("-")

    #print(new_x)
    return ' '.join(new_x)

In [5]:
get_eTrans_with_Word("CITY - - - - - - - - - - - - - - - X - X X - X X X - - - - S - - - - - - - - - - - - - -", vocabs)

'- - - - - - - - - - - - - - - CITY_X - CITY_X CITY_X - CITY_X CITY_X CITY_X - - - - CITY_S - - - - - - - - - - - - - -'

In [6]:
def numberize_sentences(sentences, vocab_idmap):  

    sentences_id=[]  

    for sid, sent in enumerate (sentences):
        tmp_list = []
        #print(sid)
        for wrd in sent.split():
            if wrd in vocab_idmap:
                wrd_id = vocab_idmap[wrd]  
            else:
                wrd_id = 0
            tmp_list.append(wrd_id)
            #print(tmp_list)
        sentences_id.append(tmp_list)

    return sentences_id 

In [7]:
def adjust_index(X, maxlen=None, window_size=3):

    if maxlen: # exclude tweets that are larger than maxlen
        new_X = []
        for x in X:

            if len(x) > maxlen:
                #print("************* Maxlen of whole dataset: " + str(len(x)) )
            	tmp = x[0:maxlen]
            	tmp[maxlen-window_size:maxlen] = ['0'] * window_size
            	new_X.append(tmp)
            else:
                new_X.append(x)

        X = new_X

    return X

In [8]:
def load_and_numberize_egrids(filelist="list_of_grid.txt", maxlen=10000, w_size=3, E=None, vocabs=None, emb_size=300, perm_num=20):
    # loading entiry-grid data from list of pos document and list of neg document
    if vocabs is None:
        print("Please input vocab list")
        return None

    list_of_files = [line.rstrip('\n') for line in open(filelist)]
    #print(list_of_files)

    # process postive gird, convert each file to be a sentence
    sentences_1 = []
    sentences_0 = []
    #print(len(list_of_files))
    for file in list_of_files:
        #print(file) 

        lines = [line.rstrip('\n') for line in open(file + ".EGrid")]
        #f_lines = [line.rstrip('\n') for line in open(file + ".Feats")]

        grid_1 = "0 "* w_size
        #print(grid_1)
        for idx, line in enumerate(lines):
            #print(line)
            e_trans = get_eTrans_with_Word(line, vocabs) # merge the grid of positive document  
            if len(e_trans) !=0:
                #print e_trans
                grid_1 = grid_1 + e_trans + " " + "0 "* w_size
        #print(grid_1)
        #print("*********************************************************")
        
        p_count = 0
        for i in range(1,perm_num+1): # reading the permuted docs
            permuted_lines = [p_line.rstrip('\n') for p_line in open(file+ ".EGrid" +"-"+str(i))]    
            grid_0 = "0 "* w_size

            for idx, p_line in enumerate(permuted_lines):
                e_trans_0 = get_eTrans_with_Word(p_line, vocabs)
                if len(e_trans_0) !=0:
                    grid_0 = grid_0 + e_trans_0  + " " + "0 "* w_size

            if grid_0 != grid_1: #check the duplication
                p_count = p_count + 1
                sentences_0.append(grid_0)
            #else:
            #    print(file+ ".EGrid" +"-"+str(i)) // print duplicates permuted docs with original
        
        for i in range (0, p_count): #stupid code
            sentences_1.append(grid_1)
        '''
        print(sentences_0)
        print("******************************************************************************")
        print("******************************************************************************")
        print("******************************************************************************")
        print("******************************************************************************")
        print("******************************************************************************")
        print(sentences_1)
        '''
    
    assert len(sentences_0) == len(sentences_1)
    
    vocab_x =[]
    for ent in vocabs:
        vocab_x.append(ent + "_S")
        vocab_x.append(ent + "_O")
        vocab_x.append(ent + "_X")

    vocab_x.append("-")
    vocab_x.append("0")
    
    #print(vocab_x)
    #print(len(vocabs))
    #print(len(vocab_x))
    
    vocab_idmap = {}
    for i in range(len(vocab_x)):
        vocab_idmap[vocab_x[i]] = i
    #print(vocab_idmap)
    #print(sentences_1)
    
    # Numberize the sentences
    X_1 = numberize_sentences(sentences_1, vocab_idmap)
    X_0  = numberize_sentences(sentences_0,  vocab_idmap)
    #print(X_1)
    
    
    X_1 = adjust_index(X_1, maxlen=maxlen, window_size=w_size)
    X_0  = adjust_index(X_0,  maxlen=maxlen, window_size=w_size)

    X_1 = sequence.pad_sequences(X_1, maxlen)
    X_0 = sequence.pad_sequences(X_0, maxlen)


    np.random.seed(2017)
    E      = 0.01 * np.random.uniform( -1.0, 1.0, (len(vocab_x), emb_size))
    E[len(vocab_x)-1] = 0
    
    return X_1, X_0, E
    

In [9]:
X_1, X_0, E = load_and_numberize_egrids(filelist="./final_data/wsj.dev", maxlen=2000, w_size=6, vocabs=vocabs)

OK


In [10]:
E

array([[-0.0095808 ,  0.0053414 , -0.0010416 , ..., -0.00538052,
        -0.00760095, -0.00241078],
       [-0.00952895, -0.00265105,  0.00043136, ..., -0.00439773,
        -0.00719359, -0.00841277],
       [ 0.00661687,  0.00533594, -0.00848318, ..., -0.00893795,
         0.00890061,  0.00605507],
       ..., 
       [-0.00239471, -0.00426782, -0.00478336, ..., -0.005811  ,
         0.00383426, -0.00889049],
       [ 0.00123934,  0.0052735 , -0.00486173, ..., -0.00560788,
        -0.00919979,  0.0041078 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [11]:
X_1

array([[17914, 17914, 17914, ...,     0,     0,     0],
       [17914, 17914, 17914, ...,     0,     0,     0],
       [17914, 17914, 17914, ...,     0,     0,     0],
       ..., 
       [    0,     0,     0, ..., 17914, 17914, 17914],
       [    0,     0,     0, ..., 17914, 17914, 17914],
       [    0,     0,     0, ..., 17914, 17914, 17914]], dtype=int32)

In [12]:
X_0

array([[17914, 17914, 17914, ...,     0,     0,     0],
       [17914, 17914, 17914, ...,     0,     0,     0],
       [17914, 17914, 17914, ...,     0,     0,     0],
       ..., 
       [    0,     0,     0, ..., 17914, 17914, 17914],
       [    0,     0,     0, ..., 17914, 17914, 17914],
       [    0,     0,     0, ..., 17914, 17914, 17914]], dtype=int32)

In [14]:
E.shape

(17915, 300)

In [65]:
#get entity transition from a row of Entity Grid
def get_eTrans_with_Feats(sent="",feats="",fn=None):
    x = sent.split()
    #print(x)
    length = len(x)
    e_occur = x.count('X') + x.count('S') + x.count('O') #counting the number of occurrence of entities
    #print(e_occur)
    if length > 80:
        if e_occur < 3:
            return ""
    elif length > 20:
        if e_occur < 2:
            return ""
    
    if fn==None: #coherence model without features
        x = x[1:]
        return ' '.join(x)     

    f = feats.split()
    #print(f)
    #print(x[0] + " -- " + f[0])
    assert f[0] == x[0] # checking working on the same entity 
    #print(x[0] + " -- " + f[0])
    #print(x)
    #print(f)
    
    x = x[1:]
    f = f[1:]
    x_f = []
    
    for sem_role in x:
        new_role = sem_role;
        if new_role != '-':
            for i in fn:
                #print(i)
                if i == 0: #adding salience
                    if e_occur == 1:
                        new_role = new_role + "F01"
                    elif e_occur == 2:
                        new_role = new_role + "F02"
                    elif e_occur == 3:
                        new_role = new_role + "F03"
                    elif e_occur >3 :
                        new_role = new_role + "F04"
                    
                else:
                    #print("hello")
                    #print(f[i-1])
                    new_role = new_role + "F" + str(i) + f[i-1] # num feat = idx + 1
  
        x_f.append(new_role)

    return ' '.join(x_f)



In [62]:
get_eTrans_with_Feats(sent='                NOV. X -', feats='                                    NOV. 0 0 1 0 0 1 0 1 1', fn=[0, 1, 3])

1


'XF01F10F31 -'

In [16]:


#get entity transition from a row of Entity Grid for insertion experiment
def get_eTrans_With_Perm(sent="",feats="",fn=None, perm=[]):
    #print(feats)
    #print(perm)

    x = sent.split()
    x = x[1:]
    length = len(x)
    e_occur = x.count('X') + x.count('S') + x.count('O') #counting the number of coocurence of the entity
    if length > 80:
        if e_occur < 3:
            return ""
    elif length > 20:
        if e_occur < 2:
            return ""
    #need to re-order the entity meaning re-order sentence in a document
    p_x = []
    for i in perm:
        p_x.append(x[i])

    if fn==None: #coherence model without features
       return ' '.join(p_x)


    f = feats.split()
    f = f[1:]

    p_x_f = []
    for sem_role in p_x: # extended coherence model without features
        new_role = sem_role;
        if new_role != '-':
            for i in fn:
                if i ==0 : #adding salience
                    if e_occur == 1:
                        new_role = new_role + "F01"
                    elif e_occur == 2:
                        new_role = new_role + "F02"
                    elif e_occur == 3:
                        new_role = new_role + "F03"
                    elif e_occur >3 :
                        new_role = new_role + "F04"
                else:
                    new_role = new_role + "F" + str(i) + f[i-1] # num feat = idx + 1
  
        p_x_f.append(new_role)

    return ' '.join(p_x_f)


In [18]:
def load_POS_EGrid(filename="", w_size=3, maxlen=1000, vocab_list=None , fn=None ):
    lines = [line.rstrip('\n') for line in open(filename + ".EGrid")]
    f_lines = [line.rstrip('\n') for line in open(filename + ".Feats")]
    #print(lines)
    #print("***********************************")
    #print(f_lines)

    grid_1 = "0 "* w_size
    for idx, line in enumerate(lines):
        # merge the grid of positive document 
        e_trans = get_eTrans_with_Feats(sent=line, feats=f_lines[idx], fn=fn)
        if len(e_trans) !=0:
            grid_1 = grid_1 + e_trans + " " + "0 "* w_size
            #print(e_trans)
    #print(grid_1)
    vocab_idmap = {}
    for i in range(len(vocab_list)):
        vocab_idmap[vocab_list[i]] = i

    X_1 = numberize_sentences([grid_1], vocab_idmap)
    X_1 = adjust_index(X_1, maxlen=maxlen, window_size=w_size)
    X_1 =  sequence.pad_sequences(X_1, maxlen)

    return X_1


In [19]:
load_POS_EGrid(filename="./final_data/EGrid.train_dev/wsj_0001.pos.text.parsed.ner")

['                NOV. X -', '              PIERRE X -', '              VINKEN S S', '               YEARS X -', '               BOARD O -', '            DIRECTOR X -', '                 MR. - X', '            CHAIRMAN - O', '            ELSEVIER - X', '                N.V. - X', '          PUBLISHING - X', '               GROUP - X']
***********************************
['                                    NOV. 0 0 1 0 0 1 0 1 1', '                                  PIERRE 0 0 1 0 0 1 0 0 0', '                                  VINKEN 0 0 1 1 0 0 0 1 1', '                                   YEARS 1 1 0 0 0 1 0 1 1', '                                   BOARD 0 1 0 0 0 1 0 1 1', '                                DIRECTOR 0 0 0 0 0 1 0 1 1', '                                     MR. 0 0 1 0 0 0 0 0 0', '                                CHAIRMAN 1 1 0 0 0 1 0 1 1', '                                ELSEVIER 0 0 1 0 0 0 0 0 0', '                                    N.V. 0 0 1 0 0 0 0 1 1', '     

TypeError: object of type 'NoneType' has no len()

In [20]:
def load_NEG_EGrid(filename="", w_size=3, maxlen=1000,  vocab_list=None, fn=None, perm=None):
    #print(perm)
    if perm != None:
        lines = [line.rstrip('\n') for line in open(filename + ".EGrid")]
        f_lines = [line.rstrip('\n') for line in open(filename + ".Feats")]


        grid_0 = "0 "* w_size
        for idx, line in enumerate(lines):
            #print(line)
            e_trans_0 = get_eTrans_With_Perm(sent=line,feats=f_lines[idx],fn=fn, perm=perm) # need to include features
            if len(e_trans_0) !=0:
                grid_0 = grid_0 + e_trans_0  + " " + "0 "* w_size
            #print(e_trans_0)
        
        #print(grid_0)
        vocab_idmap = {}
        for i in range(len(vocab_list)):
            vocab_idmap[vocab_list[i]] = i

        X_0 = numberize_sentences([grid_0], vocab_idmap)
        X_0 = adjust_index(X_0, maxlen=maxlen, window_size=w_size)
        X_0 = sequence.pad_sequences(X_0, maxlen)

        return X_0

    else:
        print("no permuted list")
        return None

In [66]:
def load_all(filelist="list_of_grid.txt",fn=None):

    list_of_files = [line.rstrip('\n') for line in open(filelist)]
    print("Using features: " + str(fn))
    vocab = Counter()

    for file in list_of_files:
#        print(file)
        lines = [line.rstrip('\n') for line in open(file + ".EGrid")]
        f_lines = [line.rstrip('\n') for line in open(file + ".Feats")]

        for idx, line in enumerate(lines):
            # merge the grid of positive document 
            e_trans = get_eTrans_with_Feats(sent=line,feats=f_lines[idx],fn=fn)
            # need to update the dictionary here
            if len(e_trans) !=0:
                for wrd in e_trans.split():
                    vocab[wrd] += 1

    vocab = dict (vocab)
    vocab_list = sorted (vocab.keys())
    vocab_list.append('0')
    print( "Total vocabulary size in the whole dataset: " + str (len(vocab)))

    return vocab_list

In [67]:
load_all(filelist="final_data/wsj.all",fn=[0,1,3])

Using features: [0, 1, 3]
Total vocabulary size in the whole dataset: 49


['-',
 'OF01F10F30',
 'OF01F10F31',
 'OF01F11F30',
 'OF01F11F31',
 'OF02F10F30',
 'OF02F10F31',
 'OF02F11F30',
 'OF02F11F31',
 'OF03F10F30',
 'OF03F10F31',
 'OF03F11F30',
 'OF03F11F31',
 'OF04F10F30',
 'OF04F10F31',
 'OF04F11F30',
 'OF04F11F31',
 'SF01F10F30',
 'SF01F10F31',
 'SF01F11F30',
 'SF01F11F31',
 'SF02F10F30',
 'SF02F10F31',
 'SF02F11F30',
 'SF02F11F31',
 'SF03F10F30',
 'SF03F10F31',
 'SF03F11F30',
 'SF03F11F31',
 'SF04F10F30',
 'SF04F10F31',
 'SF04F11F30',
 'SF04F11F31',
 'XF01F10F30',
 'XF01F10F31',
 'XF01F11F30',
 'XF01F11F31',
 'XF02F10F30',
 'XF02F10F31',
 'XF02F11F30',
 'XF02F11F31',
 'XF03F10F30',
 'XF03F10F31',
 'XF03F11F30',
 'XF03F11F31',
 'XF04F10F30',
 'XF04F10F31',
 'XF04F11F30',
 'XF04F11F31',
 '0']